# YouTube downloader and Summary Generator Application

### Installing required libraries

In [20]:
!pip install -q openai
!pip install -q pytube

### Load Environment variables

In [21]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print("API Key Loaded:", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded: True


### YouTube Downloader

In [23]:
from openai import OpenAI
from pytube import YouTube

client = OpenAI()

In [24]:
link='https://www.youtube.com/watch?v=XBq6Tm2gv-4'
video = YouTube(link)

In [25]:
print(f'Title:{video.title}')

Title:7 Powerful Hacks for Better Sleep


In [26]:
print(f'Length:{video.length}')
print(f'Number of views:{video.views}')
print(f'Thumbnail url:{video.thumbnail_url}')

Length:690
Number of views:2050
Thumbnail url:https://i.ytimg.com/vi/XBq6Tm2gv-4/hq720.jpg


In [27]:
print(video.streams)

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="244" mime_type="video/webm" res="480p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="134" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.4d401e" progressive="False" type="video">, <Stream: itag="243" mime_type="video/webm" res="360p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="133

In [31]:
youtube_stream = video.streams.get_highest_resolution()
print('Downloading your video ...')
youtube_stream.download()
print('Download is completed ...')

Download is completed ...


In [33]:
audio_file = video.streams.filter(only_audio=True).first()
print('Downloading the audio stream ...')
output_file = audio_file.download()
print('Downloading the audio stream completed ...')

In [34]:
basename = os.path.basename(output_file)
name, extension = os.path.splitext(basename)
audio_file = f'{name}.mp3'
os.rename(basename, audio_file)

In [35]:
def youtube_audio_download(link):
    from pytube import YouTube
    import os

    video = YouTube(link)
    
    audio_file = video.streams.filter(only_audio=True).first()
    print('Downloading the audio stream ...')
    output_file = audio_file.download()
    print('Downloading the audio stream completed ...')
    
    basename = os.path.basename(output_file)
    name, extension = os.path.splitext(basename)
    audio_file = f'{name}.mp3'
    os.rename(basename, audio_file)

    return audio_file

In [37]:
mp3_file = youtube_audio_download("https://www.youtube.com/watch?v=XBq6Tm2gv-4")
print(mp3_file)

7 Powerful Hacks for Better Sleep.mp3


#### Transcribing audio file using Whisper 

In [38]:
def transcribe(audio_file, not_english=False):
    if not os.path.exists(audio_file):
        print('Audio file provided does not exist!')
        return False

    if not_english:
        print('Starting translating to English ...', end='')
        transcription = client.audio.translations.create(
            model='whisper-1',
            file=open(audio_file, 'rb')
        )
    else:
        print('Starting transcription ...', end='')
        transcription = client.audio.transcriptions.create(
            model='whisper-1',
            file=open(audio_file, 'rb')
        )
        print('Transcription Completed!')

    name, extension = os.path.splitext(audio_file)
    transcription_filename=f'transcript-{name}.txt'
    with open(transcription_filename, 'w') as f:
        f.write(transcription.text)

    return transcription_filename

In [39]:
transcription_filename = transcribe(mp3_file)

Starting transcription ...Transcription Completed!


In [40]:
print(transcription_filename)

transcript-7 Powerful Hacks for Better Sleep.txt


In [41]:
with open(transcription_filename) as f:
    print(f.read())

So here's how I hacked sleep. I'm a busy person and when I sleep, I want to be able to get the most restful states of sleep in as little time as possible. You see, sleep is so important to us. It helps us burn fat. It puts us in more positive bliss states. It even improves our cognition at work. Here's a fascinating study that many of you may not have heard about but shows you just how much you're going to lose in productivity if you skip just an hour of sleep. So the U.S. military did an experiment on soldiers and what they found is that soldiers who had seven hours of sleep were 95% accurate on the range. But if they dropped their sleep by just an hour and had six hours of sleep, their accuracy fell by 50%. Think about that. If you pull an all-nighter and you skip out on an hour of sleep, your brain cognition the next day at work is going to plummet by 50%. So how do you hack sleep? How do you make sleep so effective that you wake up every day in your best states and in your best hea

#### Summarization Function Using GPT Model

In [44]:
def summarize(transcription_filename):
    if not os.path.exists(transcription_filename):
        print('The transcript file doesnot exist!')
        return False
    with open(transcription_filename) as f:
        transcript = f.read()

    system_prompt = 'I want you to act as my senior assistant'
    prompt=f'''
    Create a summary of the following text.
    Text: {transcript}

    1. Add a title to the summary.
    2. Your summary should be informative and factual, covering the most aspect of the topic.
    3. Start your summary with an introduction paragraph that gives an overview of the topic .
    4. End the summary with a conclusion phrase.
    '''

    print('Starting the summarization process ...', end='')
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages = [
            {'role':'system', 'content': system_prompt},
            {'role':'user', 'content': prompt}
        ],
        max_tokens=2048,
        temperature = 1
    )
    print('Sumamrization is completed!')
    res = response.choices[0].message.content
    return res

### Testing the applicaton

In [49]:
link = input('Input the link of the YouTube video you want to summarize:')
mp3_file = youtube_audio_download(link)
transcript_file = transcribe(mp3_file, not_english=True)
summary = summarize(transcript_file)
print('\n')
print(summary)

Input the link of the YouTube video you want to summarize: https://www.youtube.com/watch?v=K5KAc5CoCuk


Starting translating to English ...Starting the summarization process ...Sumamrization is completed!


Title: Reflections on Sweet Suffering and Resilience

Introduction: The text delves into the emotions of sweet suffering and the yearning for lost love, expressed through poetic musings on loneliness, pain, and the desire to escape. The protagonist navigates their internal turmoil while dancing through the challenges of life with a mix of love and fear, capturing the essence of resilience in the face of adversity.

Summary: The protagonist laments their insignificance without their beloved, wandering through the subway in search of solace. Seeking to forget their immense pain, they yearn to start afresh and run away from their suffering. Despite their struggles, they find moments of joy in dancing with the elements, albeit overshadowed by fear and the looming presence of pain. The narrative unfolds with a sense of surrender in Paris, ultimately culminating in a symbolic act of flying 